In [1]:
pip install pandas


Note: you may need to restart the kernel to use updated packages.


In [2]:
pip install numpy


Note: you may need to restart the kernel to use updated packages.


In [3]:
pip install scikit-learn

Note: you may need to restart the kernel to use updated packages.


In [4]:
import pandas as pd
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity

In [6]:
# Load the dataset
data = pd.read_csv('summer-products-with-rating-and-performance_2020-08.csv')

In [7]:
data.head()

,index,title,title_orig,price,retail_price,currency_buyer,units_sold,uses_ad_boosts,rating,rating_count,...,merchant_rating_count,merchant_rating,merchant_id,merchant_has_profile_picture,merchant_profile_picture,product_url,product_picture,product_id,theme,crawl_month
0,0,2020 Summer Vintage Flamingo Print Pajamas Se...,2020 Summer Vintage Flamingo Print Pajamas Se...,16.00,14,EUR,100,0,3.76,54,...,568,4.128521,595097d6a26f6e070cb878d1,0,NaN,https://www.wish.com/c/5e9ae51d43d6a96e303acdb0,https://contestimg.wish.com/api/webimage/5e9ae...,5e9ae51d43d6a96e303acdb0,summer,2020-08
1,1,SSHOUSE Summer Casual Sleeveless Soirée Party ...,Women's Casual Summer Sleeveless Sexy Mini Dress,8.00,22,EUR,20000,1,3.45,6135,...,17752,3.899673,56458aa03a698c35c9050988,0,NaN,https://www.wish.com/c/58940d436a0d3d5da4e95a38,https://contestimg.wish.com/api/webimage/58940...,58940d436a0d3d5da4e95a38,summer,2020-08
2,2,2020 Nouvelle Arrivée Femmes Printemps et Été ...,2020 New Arrival Women Spring and Summer Beach...,8.00,43,EUR,100,0,3.57,14,...,295,3.989831,5d464a1ffdf7bc44ee933c65,0,NaN,https://www.wish.com/c/5ea10e2c617580260d55310a,https://contestimg.wish.com/api/webimage/5ea10...,5ea10e2c617580260d55310a,summer,2020-08
3,3,Hot Summer Cool T-shirt pour les femmes Mode T...,Hot Summer Cool T Shirt for Women Fashion Tops...,8.00,8,EUR,5000,1,4.03,579,...,23832,4.020435,58cfdefdacb37b556efdff7c,0,NaN,https://www.wish.com/c/5cedf17ad1d44c52c59e4aca,https://contestimg.wish.com/api/webimage/5cedf...,5cedf17ad1d44c52c59e4aca,summer,2020-08
4,4,Femmes Shorts d'été à lacets taille élastique ...,Women Summer Shorts Lace Up Elastic Waistband ...,2.72,3,EUR,100,1,3.10,20,...,14482,4.001588,5ab3b592c3911a095ad5dadb,0,NaN,https://www.wish.com/c/5ebf5819ebac372b070b0e70,https://contestimg.wish.com/api/webimage/5ebf5...,5ebf5819ebac372b070b0e70,summer,2020-08


In [10]:
user_item_data = data[['merchant_id', 'rating', 'units_sold']]

In [13]:
# Select the relevant columns for collaborative filtering
user_item_data = data[['merchant_id', 'rating', 'units_sold']]

In [15]:
user_item_data = data[['merchant_id', 'rating', 'units_sold']].copy()

# Drop rows with missing values
user_item_data.dropna(inplace=True)


In [17]:
pivot_data = user_item_data.pivot_table(index='merchant_id', columns='units_sold', values='rating', aggfunc='mean')


In [18]:
# Fill missing values with 0
pivot_data.fillna(0, inplace=True)

In [20]:
# Calculate the cosine similarity matrix
similarity_matrix = cosine_similarity(pivot_data)

In [22]:
def get_recommendations(merchant_id, top_n=5):
    # Find the index of the target merchant
    target_merchant_index = pivot_data.index.get_loc(merchant_id)
    
    # Get the similarity scores for the target merchant
    target_merchant_similarities = similarity_matrix[target_merchant_index]
    
    # Sort the similarity scores in descending order
    most_similar_merchants_indices = np.argsort(target_merchant_similarities)[::-1]
    
    # Exclude the target merchant from the similar merchants list
    most_similar_merchants_indices = np.delete(most_similar_merchants_indices, 0)
    
    # Get the top-N recommendations
    recommendations = pivot_data.iloc[most_similar_merchants_indices[:top_n], :].mean(axis=0)
    
    return recommendations

# Example usage:
merchant_id = '595097d6a26f6e070cb878d1'
recommendations = get_recommendations(merchant_id)
print(recommendations)

units_sold
1         0.00
2         0.00
3         0.00
6         0.00
7         0.00
8         0.00
10        0.00
50        0.00
100       3.78
1000      0.00
5000      0.00
10000     0.00
20000     0.00
50000     0.00
100000    0.00
dtype: float64
